In [2]:
import numpy as np
import pandas as pd

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt

import pandas as pd
import requests

# Question 1

In [5]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"
response = requests.get(url, headers=headers)

ipo_filing_df = pd.read_html(response.text)

In [11]:
ipo_filing_df = ipo_filing_df[0]

In [13]:
ipo_filing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Filing Date     325 non-null    object
 1   Symbol          325 non-null    object
 2   Company Name    325 non-null    object
 3   Price Range     325 non-null    object
 4   Shares Offered  325 non-null    object
dtypes: object(5)
memory usage: 12.8+ KB


In [12]:
ipo_filing_df.head(3)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
0,"Apr 29, 2024",HWEC,"HW Electro Co., Ltd.",$3.00,3750000
1,"Apr 29, 2024",DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000
2,"Apr 26, 2024",EURK,Eureka Acquisition Corp,$10.00,5000000


In [14]:
ipo_filing_df['Filing Date'] = pd.to_datetime(ipo_filing_df['Filing Date'])

In [21]:
ipo_filing_df['Shares Offered'] = pd.to_numeric(ipo_filing_df['Shares Offered'], errors='coerce')

In [23]:
#ipo_filing_df[ipo_filing_df['Shares Offered'].isnull()]

,Filing Date,Symbol,Company Name,Price Range,Shares Offered
3,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN
4,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN
6,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,NaN
7,2024-04-15,GAUZ,Gauzy Ltd.,-,NaN
8,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN
...,...,...,...,...,...
300,2021-03-02,NSAL,"Navios South American Logistics, Inc.",-,NaN
314,2021-01-19,DECI,"Decipher Biosciences, Inc.",-,NaN
316,2020-09-08,GMWX,"GameWorks, Inc.",-,NaN
317,2020-09-04,SCIT,Sancai Holding Group Ltd.,-,NaN


In [30]:
ipo_filing_df['Price Range'] = ipo_filing_df['Price Range'].str.replace('$','')

C:\Users\nisith.singh\AppData\Local\Temp\ipykernel_36700\3171195870.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  ipo_filing_df['Price Range'] = ipo_filing_df['Price Range'].str.replace('$','')


In [43]:
ipo_filing_df['Price Range_nos'] = ipo_filing_df['Price Range'].str.split('-').apply(lambda x: len(x))

In [45]:
ipo_filing_df.loc[ipo_filing_df['Price Range'] == '-','Price Range_nos'] = 0

In [51]:
def calcAvgPrice(x):

    if x['Price Range_nos'] == 0:
        return 0
    elif x['Price Range_nos'] == 1:
        return float(x['Price Range'].split('-')[0])
    elif x['Price Range_nos'] == 2:
        return np.mean([float(i) for i in x['Price Range'].split('-')] )


In [53]:
ipo_filing_df['Avg_price'] = ipo_filing_df.apply(lambda x: calcAvgPrice(x), axis=1)

In [58]:
ipo_filing_df["Shares_offered_value"] = ipo_filing_df['Shares Offered'] * ipo_filing_df['Avg_price']

In [67]:
ipo_filing_df[(ipo_filing_df['Filing Date'].dt.year == 2023) & (ipo_filing_df['Filing Date'].dt.dayofweek==4)]['Shares_offered_value'].sum()

285700000.0

In [158]:
ipo_filing_df.to_csv("q1_output.csv")

# Question 2

In [68]:
url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipos_2023 = ipo_dfs[0]

url = "https://stockanalysis.com/ipos/2024/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)

ipos_2024 = ipo_dfs[0]

stacked_ipos_df = pd.concat([ipos_2024, ipos_2023], ignore_index=True)

stacked_ipos_df['IPO Date'] = pd.to_datetime(stacked_ipos_df['IPO Date'])

stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
# not sure why, but need to call it again to transform 'object' to 'float64'
stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'])

# Convert "Current" column
stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')

# Convert 'Return' to numeric format (percentage)
stacked_ipos_df['Return'] = pd.to_numeric(stacked_ipos_df['Return'].str.replace('%', ''), errors='coerce') / 100

C:\Users\nisith.singh\AppData\Local\Temp\ipykernel_36700\1160792438.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stacked_ipos_df['IPO Price'] = pd.to_numeric(stacked_ipos_df['IPO Price'].str.replace('$', ''), errors='coerce')
C:\Users\nisith.singh\AppData\Local\Temp\ipykernel_36700\1160792438.py:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  stacked_ipos_df['Current'] = pd.to_numeric(stacked_ipos_df['Current'].str.replace('$', ''), errors='coerce')


In [70]:
stacked_ipos_df_subst = stacked_ipos_df[stacked_ipos_df['IPO Date']<pd.to_datetime("2024-03-01")]

In [74]:
#Removing 'RYZB'
stacked_ipos_df_subst = stacked_ipos_df_subst[stacked_ipos_df_subst['Symbol']!='RYZB']
stacked_ipos_df_subst.shape

(184, 6)

In [77]:
ipo_tickers = stacked_ipos_df_subst['Symbol'].values

In [78]:
ohlcv_lst = {}

for idx,ticker in enumerate(ipo_tickers):
    print(idx)
    df_ticker = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

    ohlcv_lst[ticker] = df_ticker

0


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


1
2
3


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

4
5
6


...

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


178
179
180


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


181
182
183


[*********************100%%**********************]  1 of 1 completed


In [79]:
ohlcv_lst.keys()

dict_keys(['SMXT', 'VHAI', 'DYCQ', 'CHRO', 'UMAC', 'TBBB', 'MGX', 'HLXB', 'TELO', 'KYTX', 'PMNT', 'AHR', 'LEGT', 'ANRO', 'GUTS', 'AS', 'FBLG', 'BTSG', 'AVBP', 'HAO', 'CGON', 'YIBO', 'SUGP', 'JL', 'KSPI', 'JVSA', 'PSBD', 'CCTG', 'SYNX', 'SDHC', 'ROMA', 'IROH', 'LGCB', 'ZKH', 'BAYA', 'INHD', 'AFJK', 'GSIW', 'FEBO', 'CLBR', 'ELAB', 'RR', 'DDC', 'SHIM', 'GLAC', 'SGN', 'HG', 'CRGX', 'ANSC', 'AITR', 'GVH', 'LXEO', 'PAPL', 'ATGL', 'MNR', 'WBUY', 'NCL', 'BIRK', 'GMM', 'PMEC', 'LRHC', 'GPAK', 'SPKL', 'QETA', 'MSS', 'ANL', 'SYRA', 'VSME', 'LRE', 'TURB', 'MDBH', 'KVYO', 'CART', 'DTCK', 'NMRA', 'ARM', 'SPPL', 'NWGL', 'SWIN', 'IVP', 'NNAG', 'SRM', 'SPGC', 'LQR', 'NRXS', 'FTEL', 'MIRA', 'PXDT', 'CTNT', 'HRYU', 'SRFM', 'PRZO', 'HYAC', 'KVAC', 'JNVR', 'ELWS', 'WRNT', 'TSBX', 'ODD', 'APGE', 'NETD', 'SGMT', 'BOWN', 'SXTP', 'PWM', 'VTMX', 'INTS', 'SVV', 'KGS', 'FIHL', 'GENK', 'BUJA', 'BOF', 'AZTR', 'CAVA', 'ESHA', 'ATMU', 'ATS', 'IPXX', 'CWD', 'SGE', 'SLRN', 'ALCY', 'KVUE', 'GODN', 'TRNR', 'AACT', 'JYD',

In [102]:
for idx, ticker in enumerate(ohlcv_lst.keys()):

    df_ticker = ohlcv_lst[ticker]
    df_ticker['ticker'] = ticker
    print("ticker: ",ticker, " ticker rows: ", df_ticker.shape[0])
    if idx == 0:
        
        df_ipofiling_ohlcv = df_ticker.reset_index()
        
    else:

        df_ipofiling_ohlcv = pd.concat([df_ipofiling_ohlcv, df_ticker.reset_index() ])

    del(df_ticker)
    print("df_ipofiling_ohlcv rows :", df_ipofiling_ohlcv.shape[0])

ticker:  SMXT  ticker rows:  48
df_ipofiling_ohlcv rows : 48
ticker:  VHAI  ticker rows:  49
df_ipofiling_ohlcv rows : 97
ticker:  DYCQ  ticker rows:  16
df_ipofiling_ohlcv rows : 113
ticker:  CHRO  ticker rows:  54
df_ipofiling_ohlcv rows : 167
ticker:  UMAC  ticker rows:  56
df_ipofiling_ohlcv rows : 223
ticker:  TBBB  ticker rows:  59
df_ipofiling_ohlcv rows : 282
ticker:  MGX  ticker rows:  59
df_ipofiling_ohlcv rows : 341
ticker:  HLXB  ticker rows:  58
df_ipofiling_ohlcv rows : 399
ticker:  TELO  ticker rows:  59
df_ipofiling_ohlcv rows : 458
ticker:  KYTX  ticker rows:  59
df_ipofiling_ohlcv rows : 517
ticker:  PMNT  ticker rows:  60
df_ipofiling_ohlcv rows : 577
ticker:  AHR  ticker rows:  61
df_ipofiling_ohlcv rows : 638
ticker:  LEGT  ticker rows:  26
df_ipofiling_ohlcv rows : 664
ticker:  ANRO  ticker rows:  64
df_ipofiling_ohlcv rows : 728
ticker:  GUTS  ticker rows:  64
df_ipofiling_ohlcv rows : 792
ticker:  AS  ticker rows:  65
df_ipofiling_ohlcv rows : 857
ticker:  FBLG 

In [98]:
len(ohlcv_lst.keys())

184

In [100]:
df_ipofiling_ohlcv['ticker'].unique().shape

(183,)

In [104]:
#PTHR data was missing. It was renamed to HOVR 
ticker = 'HOVR'
df_ticker = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")
print(df_ticker.shape)
ohlcv_lst[ticker] = df_ticker
del(ohlcv_lst['PTHR'])

[*********************100%%**********************]  1 of 1 completed

(274, 6)


In [105]:
len(ohlcv_lst.keys())

184

In [106]:
df_ticker['ticker'] = ticker
df_ipofiling_ohlcv = pd.concat([df_ipofiling_ohlcv, df_ticker.reset_index() ])

In [107]:
df_ipofiling_ohlcv['ticker'].unique().shape

(184,)

In [110]:
df_ipofiling_ohlcv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37377 entries, 0 to 273
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       37377 non-null  datetime64[ns]
 1   Open       37377 non-null  float64       
 2   High       37377 non-null  float64       
 3   Low        37377 non-null  float64       
 4   Close      37377 non-null  float64       
 5   Adj Close  37377 non-null  float64       
 6   Volume     37377 non-null  float64       
 7   ticker     37377 non-null  object        
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 2.6+ MB


In [118]:
def calc_growth_future_day( x, day ):

    x = x.sort_values('Date')
    x['growth_future_'+str(day)+'d'] = x['Adj Close'].shift(-day) / x['Adj Close']
    
    return x
    

In [123]:
%%time

for day in range(30):
    df_ipofiling_ohlcv =  df_ipofiling_ohlcv.groupby('ticker').apply(lambda x: calc_growth_future_day(x, day+1)) 

<timed exec>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
<timed exec>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
<timed exec>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of

CPU times: total: 5.44 s
Wall time: 7.04 s


<timed exec>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)


In [124]:
df_ipofiling_ohlcv[df_ipofiling_ohlcv['ticker']=='AACT']

,Date,Open,High,Low,Close,Adj Close,Volume,ticker,growth_future_1d,growth_future_2d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
0,2023-06-13,10.14,10.1400,10.110,10.11,10.11,5500.0,AACT,0.999011,1.000000,...,1.005539,1.004946,1.004946,1.004946,1.005935,1.005935,1.005935,1.007913,1.007913,1.007913
1,2023-06-14,10.11,10.1300,10.095,10.10,10.10,143900.0,AACT,1.000990,1.000990,...,1.005941,1.005941,1.005941,1.006931,1.006931,1.006931,1.008911,1.008911,1.008911,1.007921
2,2023-06-15,10.11,10.1100,10.100,10.11,10.11,2061100.0,AACT,1.000000,1.000989,...,1.004946,1.004946,1.005935,1.005935,1.005935,1.007913,1.007913,1.007913,1.006924,1.007913
3,2023-06-16,10.12,10.1200,10.110,10.11,10.11,251000.0,AACT,1.000989,1.000989,...,1.004946,1.005935,1.005935,1.005935,1.007913,1.007913,1.007913,1.006924,1.007913,1.006924
4,2023-06-20,10.13,10.1300,10.110,10.12,10.12,102800.0,AACT,1.000000,1.000000,...,1.004941,1.004941,1.004941,1.006917,1.006917,1.006917,1.005929,1.006917,1.005929,1.005929
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
220,2024-04-29,10.61,10.6250,10.610,10.62,10.62,30100.0,AACT,1.000000,0.999058,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
221,2024-04-30,10.68,10.6800,10.620,10.62,10.62,556100.0,AACT,0.999058,1.000942,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
222,2024-05-01,10.64,10.6400,10.610,10.61,10.61,33100.0,AACT,1.001885,1.000943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
223,2024-05-02,10.62,10.6400,10.620,10.63,10.63,2952200.0,AACT,0.999059,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
day_x_75_pcile_lst = []
day_x_median_lst = []
day_x_mean_lst = []

for day in range(30):
    #print("day: ",day + 1)
    day_x_75_pcile_lst.append(df_ipofiling_ohlcv['growth_future_'+str(day+1)+'d'].describe()['75%'])
    day_x_median_lst.append(df_ipofiling_ohlcv['growth_future_'+str(day+1)+'d'].describe()['50%'])
    day_x_mean_lst.append(df_ipofiling_ohlcv['growth_future_'+str(day+1)+'d'].describe()['mean'])
len(day_x_75_pcile_lst)

30

In [142]:
day_x_mean_lst

[0.9995414637126476,
 0.9993289759761143,
 0.9992222362947901,
 0.9990344793124465,
 0.9990181791570804,
 0.9990693705576126,
 0.9988642370551901,
 0.9987282761959282,
 0.9986114204362183,
 0.9983909012593254,
 0.9981289638490025,
 0.9977231148367675,
 0.9970877984197716,
 0.9964115843538572,
 0.9959040845562891,
 0.9955579594114955,
 0.9952534926287993,
 0.9949357703717291,
 0.9947285653513092,
 0.9945580918653487,
 0.9942096279622457,
 0.9938388806948991,
 0.9933766416377019,
 0.9929968025749683,
 0.9926912307085048,
 0.9926158186800828,
 0.9924891843009032,
 0.9922605401992468,
 0.9921319645106228,
 0.992141717620871]

In [143]:
df_ipofiling_ohlcv.isnull().sum()

Date                    0
Open                    0
High                    0
Low                     0
Close                   0
Adj Close               0
Volume                  0
ticker                  0
growth_future_1d      184
growth_future_2d      368
growth_future_3d      552
growth_future_4d      736
growth_future_5d      920
growth_future_6d     1104
growth_future_7d     1288
growth_future_8d     1472
growth_future_9d     1656
growth_future_10d    1840
growth_future_11d    2024
growth_future_12d    2208
growth_future_13d    2392
growth_future_14d    2576
growth_future_15d    2760
growth_future_16d    2944
growth_future_17d    3127
growth_future_18d    3310
growth_future_19d    3493
growth_future_20d    3676
growth_future_21d    3859
growth_future_22d    4042
growth_future_23d    4225
growth_future_24d    4408
growth_future_25d    4591
growth_future_26d    4774
growth_future_27d    4956
growth_future_28d    5138
growth_future_29d    5320
growth_future_30d    5502
dtype: int64

In [139]:
df_ipofiling_ohlcv.describe()

,Open,High,Low,Close,Adj Close,Volume,growth_future_1d,growth_future_2d,growth_future_3d,growth_future_4d,...,growth_future_21d,growth_future_22d,growth_future_23d,growth_future_24d,growth_future_25d,growth_future_26d,growth_future_27d,growth_future_28d,growth_future_29d,growth_future_30d
count,37377.000000,37377.000000,37377.000000,37377.000000,37377.000000,3.737700e+04,37193.000000,37009.000000,36825.000000,36641.000000,...,33518.000000,33335.000000,33152.000000,32969.000000,32786.000000,32603.000000,32421.000000,32239.000000,32057.000000,31875.000000
mean,12.264412,12.872159,11.820162,12.244424,12.216767,6.710233e+05,0.999541,0.999329,0.999222,0.999034,...,0.994210,0.993839,0.993377,0.992997,0.992691,0.992616,0.992489,0.992261,0.992132,0.992142
std,37.896620,56.824212,32.680708,40.589890,40.578475,5.156670e+06,0.095803,0.135382,0.166530,0.191195,...,0.433097,0.443464,0.453291,0.462350,0.472677,0.486147,0.499691,0.516723,0.536434,0.558915
min,0.002000,0.002000,0.002000,0.002000,0.002000,0.000000e+00,0.028674,0.003584,0.002729,0.002729,...,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729,0.002729
25%,1.850000,1.960000,1.742000,1.840000,1.835000,6.100000e+03,0.977539,0.966319,0.957447,0.949367,...,0.840243,0.835294,0.830492,0.823529,0.820073,0.814001,0.810811,0.806605,0.802703,0.798714
50%,7.910000,8.190000,7.600000,7.877000,7.860853,5.100000e+04,1.000000,1.000000,1.000000,1.000000,...,0.996904,0.996226,0.996135,0.995687,0.995886,0.995059,0.994644,0.994376,0.994048,0.994001
75%,12.000000,12.320000,11.510000,11.970000,11.960000,2.259000e+05,1.014778,1.020007,1.024946,1.028006,...,1.057643,1.058621,1.059192,1.060221,1.060865,1.060446,1.062751,1.064606,1.066524,1.067691
max,3069.000000,7500.000000,875.000000,4318.000000,4318.000000,3.723413e+08,6.826923,6.971311,7.625593,8.650538,...,12.634409,11.491442,12.486842,12.284789,12.661000,15.025219,14.782093,19.655310,22.918785,27.198465


In [156]:
df_ipofiling_ohlcv.to_csv("q2_output.csv")

# Question 3

In [144]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

In [145]:
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [147]:
len(LARGEST_STOCKS) , len(LARGE_STOCKS)

(33, 34)

In [150]:
ALL_TICKERS = LARGEST_STOCKS + LARGE_STOCKS
len(ALL_TICKERS)

67

In [151]:
import time

stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ALL_TICKERS):
  print(i,ticker)

  # Work with stock prices
  historyPrices = yf.download(tickers = ticker,
                     period = "max",
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [7]: #1,3,7,30,90,365
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Adj Close'] / historyPrices['Adj Close'].shift(i)
  
  #historyPrices['growth_future_5d'] = historyPrices['Adj Close'].shift(-5) / historyPrices['Adj Close']
  # # Technical indicators
  # # SimpleMovingAverage 10 days and 20 days
  # historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  # historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  # historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  # historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Adj Close']

  # # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  # historyPrices['volatility'] =   historyPrices['Adj Close'].rolling(30).std() * np.sqrt(252)

  # # what we want to predict
  # historyPrices['is_positive_growth_5d_future'] = np.where(historyPrices['growth_future_5d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 MSFT


[*********************100%%**********************]  1 of 1 completed


1 AAPL


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2 GOOG


...

[*********************100%%**********************]  1 of 1 completed


65 KOTAKBANK.NS


[*********************100%%**********************]  1 of 1 completed


66 TITAN.NS


[*********************100%%**********************]  1 of 1 completed


In [155]:
#stocks_df.to_csv('q3_input.csv')

In [196]:
stocks_df = pd.read_csv('q3_input.csv')

In [197]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])

In [198]:
stocks_df.shape

(495188, 13)

In [199]:
stocks_df = stocks_df[stocks_df['Date'].between(pd.to_datetime('2014-01-01'),pd.to_datetime('2023-12-31'))]
stocks_df.shape

(164607, 13)

In [200]:
def get_ticker_type(ticker:str, US_STOCKS, EU_STOCKS, INDIA_STOCKS, NEW_US, NEW_EU, NEW_INDIA):
  if ticker in US_STOCKS:
    return 'US_STOCKS'
  elif ticker in EU_STOCKS:
    return 'EU_STOCKS'
  elif ticker in INDIA_STOCKS:
    return 'INDIA_STOCKS'
  elif ticker in NEW_US:
    return 'NEW_US'
  elif ticker in NEW_EU:
    return 'NEW_EU'
  elif ticker in NEW_INDIA:
    return 'NEW_INDIA'
  else:
    return 'ERROR'

stocks_df['ticker_type'] = stocks_df.Ticker.apply(lambda x:get_ticker_type(x, US_STOCKS, EU_STOCKS, INDIA_STOCKS, NEW_US, NEW_EU, NEW_INDIA))

In [201]:
def get_ticker_category(ticker:str, LARGEST_STOCKS, LARGE_STOCKS):
  if ticker in LARGEST_STOCKS:
    return 'LARGEST_STOCKS'
  elif ticker in LARGE_STOCKS:
    return 'LARGE_STOCKS'
  else:
    return 'ERROR'

stocks_df['ticker_category'] = stocks_df.Ticker.apply(lambda x:get_ticker_category(x, LARGEST_STOCKS, LARGE_STOCKS))

In [202]:
stocks_df.ticker_category.value_counts()

LARGEST_STOCKS    83323
LARGE_STOCKS      81284
Name: ticker_category, dtype: int64

In [203]:
stocks_df['Date'].max(), stocks_df['Date'].min()

(Timestamp('2023-12-29 00:00:00'), Timestamp('2014-01-01 00:00:00'))

In [204]:
stocks_df.groupby(['ticker_category', 'Date']).agg({'growth_7d':'mean'})

growth_7d
ticker_category Date                 
LARGEST_STOCKS  2014-01-01   1.011797
                2014-01-02   1.006979
                2014-01-03   0.999562
                2014-01-06   0.994459
                2014-01-07   0.992542
...                               ...
LARGE_STOCKS    2023-12-22   1.003170
                2023-12-26   1.013505
                2023-12-27   1.003030
                2023-12-28   0.999068
                2023-12-29   0.999590

[5190 rows x 1 columns]

In [205]:
stocks_df

,Unnamed: 0,Open,High,Low,Close,Adj Close,Volume,Ticker,Year,Month,Weekday,Date,growth_7d,ticker_type,ticker_category
7011,7011,37.349998,37.400002,37.099998,37.160000,31.233074,30632200,MSFT,2014,1,3,2014-01-02,1.009783,US_STOCKS,LARGEST_STOCKS
7012,7012,37.200001,37.220001,36.599998,36.910000,31.022943,31134800,MSFT,2014,1,4,2014-01-03,1.007920,US_STOCKS,LARGEST_STOCKS
7013,7013,36.849998,36.889999,36.110001,36.130001,30.367353,43603700,MSFT,2014,1,0,2014-01-06,0.974380,US_STOCKS,LARGEST_STOCKS
7014,7014,36.330002,36.490002,36.209999,36.410000,30.602684,35802800,MSFT,2014,1,1,2014-01-07,0.972489,US_STOCKS,LARGEST_STOCKS
7015,7015,36.000000,36.139999,35.580002,35.759998,30.056360,59971700,MSFT,2014,1,2,2014-01-08,0.958971,US_STOCKS,LARGEST_STOCKS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495101,495101,3580.000000,3638.449951,3560.550049,3627.350098,3627.350098,777099,TITAN.NS,2023,12,4,2023-12-22,1.006940,NEW_INDIA,LARGE_STOCKS
495102,495102,3635.000000,3665.000000,3623.449951,3656.699951,3656.699951,526101,TITAN.NS,2023,12,1,2023-12-26,1.018182,NEW_INDIA,LARGE_STOCKS
495103,495103,3668.000000,3695.000000,3645.000000,3689.250000,3689.250000,666625,TITAN.NS,2023,12,2,2023-12-27,1.024635,NEW_INDIA,LARGE_STOCKS
495104,495104,3699.899902,3737.000000,3680.699951,3715.100098,3715.100098,1033648,TITAN.NS,2023,12,3,2023-12-28,1.026384,NEW_INDIA,LARGE_STOCKS


In [209]:
df_q3 = pd.pivot_table(stocks_df, values='growth_7d', index=['Date'],
                       columns=['ticker_category'], aggfunc="mean")
df_q3.shape

(2595, 2)

In [210]:
df_q3.head(1)

ticker_category,LARGEST_STOCKS,LARGE_STOCKS
Date,,
2014-01-01,1.011797,1.011684


In [211]:
df_q3['LG_GT_THAN_LGST'] = df_q3['LARGE_STOCKS'] > df_q3['LARGEST_STOCKS']

In [213]:
df_q3['LG_GT_THAN_LGST'].sum() / df_q3['LG_GT_THAN_LGST'].shape[0]

0.4705202312138728